In [1]:
import yfinance as yf
import pandas as pd

# 下載NVDA的股票數據
nvda = yf.Ticker("NVDA")
nvda_history = nvda.history(period="1y")

# 保存數據到CSV文件
csv_path = "nvda_stock_data.csv"
nvda_history[['Close']].to_csv(csv_path, index=True, index_label='Date')

print(f"CSV文件已生成：{csv_path}")

CSV文件已生成：nvda_stock_data.csv


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# 讀取數據
data = pd.read_csv('nvda_stock_data.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data = data[['Close']]

# 數據預處理
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# 生成訓練和測試數據集
train_data_len = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_data_len]
test_data = scaled_data[train_data_len:]

x_train, y_train = [], []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[1], x_train.shape[1], 1))

# 構建LSTM模型
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

# 編譯和訓練模型
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=1, epochs=1)

# 準備測試數據集
x_test = []
y_test = test_data[60:]

for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# 預測股價
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# 可視化結果
train = data[:train_data_len]
valid = data[train_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16, 8))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close Price USD')
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

ImportError: cannot import name 'path_to_string' from 'keras.utils.io_utils' (c:\Users\jk121\.conda\envs\MachineLearing\lib\site-packages\keras\utils\io_utils.py)